In [1]:
import tensorflow as tf
import numpy as np
import os

In [72]:
class WaveNet_model():

    def __init__(self, nb_layers:int, nb_filters:int):
        
        self.nb_filters= nb_filters
        audio_length = 256 # à supprimer/modifier en fonction de la longueur des échantillions audio !!!!!

        #à compléter
        inputWN = tf.keras.Input(shape = (audio_length, 256), name="WN Input")

        out = tf.keras.layers.Conv1D(filters=nb_filters,kernel_size=2, padding="causal", name='initial_causal_convolution')(inputWN)

        skip_connections = []
        for k in range(nb_layers):
            out, skipx = self.__residual_block(out)
            skip_connections.append(skipx)

        out = tf.keras.layers.Add()(skip_connections)
        out = tf.keras.layers.Activation('relu')(out)
        out = tf.keras.layers.Conv1D(filters=256, kernel_size=1, padding = "same")(out)
        out = tf.keras.layers.Activation('relu')(out)
        out = tf.keras.layers.Conv1D(filters=256, kernel_size=1, padding = "same")(out)


        outputWN =  tf.keras.layers.Activation('softmax', name="WN_Output")(out)

        self.model = tf.keras.models.Model(inputs = inputWN, outputs = outputWN)
        
        # return WNmodel


    def __delated_conv(self, input, nb_dilatation : int) -> tf.keras.layers.Conv1D :
        for i in range(nb_dilatation):
            delated_conv_f = tf.keras.layers.Conv1D(dilation_rate = 2**i) (delated_conv_f)


    def __residual_block(self,x):
        # x : output of the 
        # delated conv filter
        delated_conv_f0 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2, dilation_rate = 2**0)(x)
        delated_conv_f1 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**1)(delated_conv_f0)
        delated_conv_f2 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**2)(delated_conv_f1)
        delated_conv_f3 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**3)(delated_conv_f2)
        delated_conv_f4 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**4)(delated_conv_f3)
        delated_conv_f5 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**5)(delated_conv_f4)
        delated_conv_f6 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**6)(delated_conv_f5)
        delated_conv_f7 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**7)(delated_conv_f6)
        delated_conv_f8 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**8)(delated_conv_f7)
        delated_conv_f9 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**9)(delated_conv_f8)
        tanh_out = tf.keras.layers.Activation('tanh')(delated_conv_f9)

         # delated conv gate

        delated_conv_g0 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2, dilation_rate = 2**0)(x)
        delated_conv_g1 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**1)(delated_conv_g0)
        delated_conv_g2 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**2)(delated_conv_g1)
        delated_conv_g3 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**3)(delated_conv_g2)
        delated_conv_g4 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**4)(delated_conv_g3)
        delated_conv_g5 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**5)(delated_conv_g4)
        delated_conv_g6 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**6)(delated_conv_g5)
        delated_conv_g7 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**7)(delated_conv_g6)
        delated_conv_g8 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**8)(delated_conv_g7)
        delated_conv_g9 = tf.keras.layers.Conv1D(filters=self.nb_filters,padding = "causal", kernel_size = 2,dilation_rate = 2**9)(delated_conv_g8)
        sigmoid_out = tf.keras.layers.Activation('sigmoid')(delated_conv_g9)

        # merge x tanh & sigmoid
        multiply = tf.keras.layers.Multiply()([tanh_out, sigmoid_out])

        # conv 1,1
        resx = tf.keras.layers.Conv1D(filters=self.nb_filters,kernel_size=1, padding='same')(multiply)
        skipx = tf.keras.layers.Conv1D(filters=self.nb_filters,kernel_size=1, padding='same')(multiply)
        resx = tf.keras.layers.Add()([x, resx])

        return resx, skipx
        
    


In [73]:
WNModel = WaveNet_model(nb_layers=2, nb_filters=256)

In [75]:
WNModel.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 WN Input (InputLayer)          [(None, 256, 256)]   0           []                               
                                                                                                  
 initial_causal_convolution (Co  (None, 256, 256)    131328      ['WN Input[0][0]']               
 nv1D)                                                                                            
                                                                                                  
 conv1d_311 (Conv1D)            (None, 256, 256)     131328      ['initial_causal_convolution[0][0
                                                                 ]']                              
                                                                                            